In [13]:
# Import Dependencies
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [75]:
#declare columns
columns = ["Year","CBSA","Median AQI"]

In [85]:
# Read data from csv
full_df = pd.read_csv("full_df.csv")

In [86]:
# Display full_df
full_df.head(10)

,CBSA,CBSA Code,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,"Aberdeen, SD",10100,2012,123,104,19,0,0,0,0,77,53,28,0,0,0,109,14
1,"Aberdeen, WA",10140,2012,366,348,18,0,0,0,0,81,42,20,0,0,0,366,0
2,"Adjuntas, PR",10260,2012,77,74,3,0,0,0,0,66,36,18,0,0,0,77,0
3,"Adrian, MI",10300,2012,243,142,76,21,4,0,0,187,101,48,0,0,174,69,0
4,"Akron, OH",10420,2012,366,186,173,6,1,0,0,164,78,50,81,0,98,187,0
5,"Albany, GA",10500,2012,360,254,104,2,0,0,0,124,63,38,0,0,0,359,1
6,"Albany, OR",10540,2012,366,319,47,0,0,0,0,91,53,23,0,0,0,366,0
7,"Albany-Schenectady-Troy, NY",10580,2012,366,294,64,8,0,0,0,143,60,39,1,0,272,93,0
8,"Albuquerque, NM",10740,2012,366,100,241,23,1,1,0,291,96,60,0,2,228,105,31
9,"Alexandria, LA",10780,2012,362,177,185,0,0,0,0,88,66,52,0,0,0,362,0


In [78]:
full_df.dtypes

CBSA                                   object
CBSA Code                               int64
Year                                    int64
Days with AQI                           int64
Good Days                               int64
Moderate Days                           int64
Unhealthy for Sensitive Groups Days     int64
Unhealthy Days                          int64
Very Unhealthy Days                     int64
Hazardous Days                          int64
Max AQI                                 int64
90th Percentile AQI                     int64
Median AQI                              int64
Days CO                                 int64
Days NO2                                int64
Days Ozone                              int64
Days PM2.5                              int64
Days PM10                               int64
dtype: object

In [79]:
full_df.columns

Index(['CBSA', 'CBSA Code', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days PM2.5', 'Days PM10'],
      dtype='object')

In [80]:
# Check Record Count (should = 5680 rows)
len(full_df.index)

5680

In [81]:
# Determine Top 5 Worst AQI Cities
Worst_AQI = full_df.groupby(["Year","CBSA"])["Median AQI"].mean().sort_values(ascending=False)
Worst_AQI.head(5)

Year  CBSA                                
2021  Phoenix-Mesa-Scottsdale, AZ             123.0
2020  Phoenix-Mesa-Scottsdale, AZ             119.0
2012  Riverside-San Bernardino-Ontario, CA    100.0
2020  Riverside-San Bernardino-Ontario, CA     98.0
2017  Riverside-San Bernardino-Ontario, CA     98.0
Name: Median AQI, dtype: float64

In [82]:
# Determine Top 5 Best AQI Cities
Best_AQI = full_df.groupby(["Year","CBSA"])["Median AQI"].mean().sort_values(ascending=True)
Best_AQI.head(5)

Year  CBSA          
2022  Moscow, ID        0.0
      Danville, VA      3.0
      Pierre, SD        3.0
2021  Danville, VA      3.0
2013  Dodge City, KS    6.0
Name: Median AQI, dtype: float64

In [64]:
#view as dataframe
combined_AQI = pd.DataFrame(Worst_AQI)
combined_AQI.head(0)

,,Median AQI
Year,CBSA,


In [65]:
combined_AQI.describe()

,Median AQI
count,5680.000000
mean,38.313204
std,10.957150
min,0.000000
25%,34.000000
50%,39.000000
75%,44.000000
max,123.000000


In [66]:
combined_AQI.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5680 entries, (2021, 'Phoenix-Mesa-Scottsdale, AZ') to (2022, 'Moscow, ID')
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Median AQI  5680 non-null   float64
dtypes: float64(1)
memory usage: 65.5+ KB


In [92]:
# Break into targets and features
X = combined_AQI['Median AQI']
#y = combined_AQI['Year']
y = pd.get_dummies(combined_AQI=='Year')

In [97]:
# Check the balance of our target values
y.value_counts()

Median AQI
False         5680
dtype: int64

In [95]:
#import linear regression dependencies
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(4260,)

In [96]:
#linear regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[43. 30. 43. ... 35. 42. 35.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
#print the coefficients of the model
print('Coefficients: \n', lin_reg.coef_)

In [ ]:
#predict X_test set from data
predictions = lin_reg.predict(X_test)

In [ ]:
#real test values versus predicted values
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
#evaluate the model
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#check for distribution
plt.hist((y_test-predictions),bins=50);

In [ ]:
#review the coefficients
coeffecients = pd.DataFrame(lin_reg.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients